In [ ]:
%pip install typing_extensions

: 

In [ ]:
%pip install tensorflow
%pip install keras
%pip install tqdm

: 

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
image_dir = "../data/images"
mask_dir = "../data/masks"

image_size = (256, 256)
all_images = []
all_masks = []

for filename in os.listdir(image_dir):
    image = cv2.imread(os.path.join(image_dir, filename), cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, image_size)
    all_images.append(image)

    mask = cv2.imread(os.path.join(mask_dir, filename), cv2.IMREAD_GRAYSCALE)
    mask = cv2.resize(mask, image_size)
    all_masks.append(mask)


In [ ]:
X = np.array(all_images).reshape(-1, 256, 256, 1) / 255.0
y = np.array(all_masks).reshape(-1, 256, 256, 1) / 255.0

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)


X_train: (9637, 256, 256, 1)
y_train: (9637, 256, 256, 1)


In [9]:
!pip show tensorflow

Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\Users\Faiyaz\Desktop\SideProjects\PneumoScope\venv\Lib\site-packages
Requires: tensorflow-intel
Required-by: 


In [3]:
from tensorflow.keras import layers, models
# 1. Defining reusable conv block
def conv_block(inputs, num_filters):
    x = layers.Conv2D(num_filters, (3, 3), padding='same',
                      activation='relu')(inputs)
    x = layers.Conv2D(num_filters, (3, 3),
                      padding='same', activation='relu')(x)
    return x

ModuleNotFoundError: No module named 'tensorflow.compat'

In [ ]:
# 2. Encoder path (downsampling)
# This part of the model extracts features from the input image
# by applying a series of convolutional layers followed by max pooling
inputs = layers.Input((256, 256, 1)) # shape of greyscale chest xrays

c1 = conv_block(inputs, 64) # 64 filters in first conv block
p1 = layers.MaxPooling2D((2,2))(c1) # downsample by 2x (halves the image size by pooling)

c2 = conv_block(p1, 128) # 128 filters in second conv block
p2 = layers.MaxPooling2D((2,2))(c2) # downsample by 2x

c3 = conv_block(p2, 256) # 256 filters in third conv block
p3 = layers.MaxPooling2D((2,2))(c3) # downsample by 2x

c4 = conv_block(p3, 512) # 512 filters in fourth conv block
p4 = layers.MaxPooling2D((2,2))(c4) # downsample by 2x

In [ ]:
# 3. Bottleneck layer
# This is the deepest layer of the model, where the most abstract features are learned
bottleneck = conv_block(p4, 1024) # 1024 filters in bottleneck conv block

In [ ]:
#4. Decoder path (upsampling)
# The decoder path mirrors the encoder path, using transposed convolutions to upsample the feature maps
# and concatenate them with the corresponding feature maps from the encoder path
u1 = layers.UpSampling2D((2,2))(bottleneck) # upsample by 2x
u1 = layers.Concatenate()([u1, c4]) # concatenate with encoder feature map
c5 = conv_block(u1, 512) # 512 filters in fifth conv block

u2 = layers.UpSampling2D((2,2))(c5) # upsample by 2x
u2 = layers.Concatenate()([u2, c3]) # concatenate with encoder feature map
c6 = conv_block(u2, 256) # 256 filters in sixth conv block

u3 = layers.UpSampling2D((2,2))(c6) # upsample by 2x
u3 = layers.Concatenate()([u3, c2]) # concatenate with encoder feature map
c7 = conv_block(u3, 128) # 128 filters in seventh conv block

u4 = layers.UpSampling2D((2,2))(c7) # upsample by 2x
u4 = layers.Concatenate()([u4, c1]) # concatenate with encoder feature map
c8 = conv_block(u4, 64) # 64 filters in eighth conv block

In [ ]:
# 5. Output layer
# The final layer of the model produces the segmentation mask
output = layers.Conv2D(1, (1,1), activation='sigmoid')(c8) # final output layer with 1 filter (binary segmentation)

In [ ]:
# 6. Creating the model
# The model is created by specifying the input and output layers
model = models.Model(inputs=inputs, outputs=output) # create the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # compile the model with Adam optimizer and binary crossentropy loss

In [ ]:
import cv2

def load_data(image_folder, mask_folder, target_size=(256, 256)):
    images = []
    masks = []
    for img_name in os.listdir(image_folder):
        img_path = os.path.join(image_folder, img_name)
        mask_path = os.path.join(mask_folder, img_name)  # Ensure masks are named similarly
        
        if os.path.exists(mask_path):  # Check if corresponding mask exists
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read the image in grayscale
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read the mask in grayscale
            
            if img is not None and mask is not None:  # Ensure image is valid
                img_resized = cv2.resize(img, target_size)  # Resize the image
                mask_resized = cv2.resize(mask, target_size)  # Resize the mask
                
                images.append(img_resized)
                masks.append(mask_resized)
    
    print(f"Loaded {len(images)} images and {len(masks)} masks")  # Add this to see how many files are loaded
    return np.array(images), np.array(masks)

# Use the function with resized images
image_folder = '../data/images'  # path to your images
mask_folder = '../data/masks'  # path to your masks
X, y = load_data(image_folder, mask_folder)


In [ ]:
print("Splitting...")

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split

# Verify the size of X and y
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")

# Make sure you have enough data before splitting
if len(X) > 1:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
else:
    print("Not enough data to split into train/test sets.")

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
from keras.src.callbacks import ModelCheckpoint
import tensorflow as tf

class LivePlotCallback(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []

    def on_epoch_end(self, epoch, logs=None):
        self.losses.append(logs["loss"])
        self.val_losses.append(logs["val_loss"])
        self.acc.append(logs["accuracy"])
        self.val_acc.append(logs["val_accuracy"])

        clear_output(wait=True)
        plt.figure(figsize=(12, 5))

        # Loss plot
        plt.subplot(1, 2, 1)
        plt.plot(self.losses, label="Train Loss")
        plt.plot(self.val_losses, label="Val Loss")
        plt.title("Loss")
        plt.legend()

        # Accuracy plot
        plt.subplot(1, 2, 2)
        plt.plot(self.acc, label="Train Acc")
        plt.plot(self.val_acc, label="Val Acc")
        plt.title("Accuracy")
        plt.legend()

        plt.show()


In [ ]:
checkpoint = ModelCheckpoint(
    "../model_checkpoints/unet_best.h5",  # Save to this path
    monitor="val_loss",                # Check validation loss
    save_best_only=True,              # Only save if it's the best
    save_weights_only=False,          # Save the full model
    verbose=1                          # Print info when saving
)

live_plot = LivePlotCallback()

from keras.src.callbacks import EarlyStopping
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

from keras.src.callbacks import ReduceLROnPlateau
lr_reduce = ReduceLROnPlateau(factor=0.1, patience=3)

%pip install tensorboard
from keras.src.callbacks import TensorBoard

# Setup TensorBoard callback
%load_ext tensorboard
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1)

print("✅ Starting model training...")

# 7. Training the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=8,
    epochs=50,
    callbacks=[checkpoint, live_plot, early_stop, lr_reduce, tensorboard]
)


In [ ]:
# 8. Visualizing the training process
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')